In [2]:
# don't touch this cell
import os
from gymirdrl import ROOT_DIR
from gymirdrl.core.args import GymirArgs
from gymirdrl.core.drl_manager import GymirDrlManager
from gymirdrl.utils.parse_utils import parse_drl_args_to_gymir_args

In [6]:
# FIXED ARGS
args_dict = {
    'sim_path': os.path.join(os.path.join(os.path.dirname(ROOT_DIR), "simulations"), "baseDrlBg/omnetpp.ini"),
    'sim_port': 7895,
    'runs': 1,
    'update_period': 1.0,
    'time_limit': 512,
    'streams_cfg': "/opt/omnetpp-workspace/gymir5g/experiments/arcs23/stream_cfg.json",
    'mode': "eval",
    'env_cfg': "/opt/omnetpp-workspace/gymir5g/experiments/arcs23/env_cfg.json",
    'model_name': "sac",
    'model_file': "/opt/omnetpp-workspace/gymir5g/experiments/arcs23/model.zip",
    'is_save_drl_log': True,
    'is_save_sim_log': True,
}

args = parse_drl_args_to_gymir_args(args_dict)
RESULTS_DIR = "/opt/omnetpp-workspace/gymir5g/experiments/arcs23/results"

# Validation runs in the ARCS 23 paper:

# EASY: SameRTCDirectNoFec, run 2
# MODERATE: SameRTCDirectNoFec, run 37
# HARD: HardRTCDirectNoFec, run 7

def test_drl(args: GymirArgs, logdir: str = "") -> None:
    manager = GymirDrlManager(args, save_log_path=os.path.join(RESULTS_DIR, logdir))
    manager.eval()
    manager.env.close()
    
def test_gcc(scenario: str, from_run: int, logdir: str = "") -> None:
    os.chdir('/opt/omnetpp-workspace/gymir5g/simulations/baseDrlBg')
    os.system(f"./run -c {scenario} -r 1 -f {from_run} -sc {args.streams_cfg} -t 512 -ad abr_gcc -l {os.path.join(RESULTS_DIR, logdir)}")


In [ ]:
# validation runs

args.sim_port = 6000

## EASY DET
args.scenario = "SameRTCDirectNoFec"
args.from_run = 2
args.deterministic = True
test_drl(args, "easy_drl_det")

## EASY NDET
args.deterministic = False
test_drl(args, "easy_drl_ndet")

# EASY GCC
test_gcc("SameRTCDirectNoFec", 2, "easy_gcc")

In [ ]:
args.sim_port = 6100

## MODERATE DET
args.scenario = "SameRTCDirectNoFec"
args.from_run = 37
args.deterministic = True
test_drl(args, "mod_drl_det")

## MODERATE NDET
args.deterministic = False
test_drl(args, "mod_drl_ndet")

# MODERATE GCC
test_gcc("SameRTCDirectNoFec", 37, "mod_gcc")

In [ ]:
args.sim_port = 6200

## HARD DET
args.scenario = "HardRTCDirectNoFec"
args.from_run = 7
args.deterministic = True
test_drl(args, "hard_drl_det")

## HARD NDET
args.deterministic = False
test_drl(args, "hard_drl_ndet")

# HARD GCC
test_gcc("HardRTCDirectNoFec", 7, "hard_gcc")